In [1]:
import psycopg2
# import sqlalchemy as sq
import pandas as pd
import numpy as np
import mariadb
# import dask.dataframe as dd
import json
import os
import shutil
import subprocess
from pathlib import Path
import pyodbc

In [2]:
conn_lims = pyodbc.connect("Driver={SQL Server};"
                            "Server=192.168.5.18\CROPNUT;"
                            "Database=cropnuts;"
                            "uid=thomasTsuma;pwd=GR^KX$uRe9#JwLc6")

In [3]:
_ = pd.read_sql("select TOP(1000) * from SampleResults where group_name='Soil Analysis' AND analysis_name LIKE '%Complete Soil Analysis%'",con=conn_lims)

C:\Users\tommy\AppData\Roaming\Python\Python38\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [4]:
_.to_csv("sl.csv")

In [5]:
soil_df = pd.read_csv("input/soil_analysis_cleaned.csv")

In [6]:
len(soil_df)

4038356

In [7]:
converted_samples = [i.strip() for i in pd.read_csv("output/soil_single_analysis.csv", usecols=["sample_code"])["sample_code"].values ]

In [8]:
len(converted_samples)

75041

In [9]:
_df = pd.read_csv("output/soil_single_analysis.csv")
ct = 0
for index, row in soil_df.loc[~(soil_df['sample_code'].isin(converted_samples))].iterrows():
    ct+=1
    analyses = row['analysis_name'].split(',')
    sample_code = row['sample_code']
    
    for analysis in analyses:
        _ = pd.DataFrame(row).T
        _['analysis_name'] = analysis
        _df = pd.concat([_df, _])
    print(ct)
    _df.to_csv("output/soil_single_analysis.csv")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


MemoryError: Unable to allocate 3.66 MiB for an array with shape (3, 10000) and data type <U32

In [51]:
soil_df = pd.read_csv("output/soil_single_analysis.csv")

In [52]:
soil_df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
0,0,0,0,CF045SA0784,2012-07-09 00:00:00.000,% Organic Matter (OM),iron,165.00,ppm
1,1,0,0,CF045SA0784,2012-07-09 00:00:00.000,Complete Soil Analysis,iron,165.00,ppm
2,2,1,1,CF045SA0784,2012-07-09 00:00:00.000,% Organic Matter (OM),cec,16.60,meq/100g
3,3,1,1,CF045SA0784,2012-07-09 00:00:00.000,Complete Soil Analysis,cec,16.60,meq/100g
4,4,2,2,CF045SA0784,2012-07-09 00:00:00.000,% Organic Matter (OM),aluminium,481.00,ppm
...,...,...,...,...,...,...,...,...,...
58065,45168,23191,23195,CW017SA7887,2012-05-18 00:00:00.000,AFSIS Std Wet Chemistry Soil Analysis,ph,4.74,None
58066,45169,23192,23196,CW017SA7887,2012-05-18 00:00:00.000,AFSIS Std Wet Chemistry Soil Analysis,exchangeable_acidity,2.16,meq/100g
58067,45170,23193,23197,CW017SA7887,2012-05-18 00:00:00.000,AFSIS Std Wet Chemistry Soil Analysis,phosphorus,5.24,ppm
58068,45171,23194,23198,CW017SA7887,2012-05-18 00:00:00.000,AFSIS Std Wet Chemistry Soil Analysis,potassium,56.00,ppm


In [53]:
soil_df = soil_df.dropna(subset="result")

In [54]:
df_ = soil_df[['analysis_name','chemical_name','unit_name']].value_counts()
df_.to_csv("output/soil_unit_per_chemical.csv")

In [55]:
df_ = df_.reset_index()

In [56]:
crop_chem_unit_df = pd.DataFrame()
for crop in df_['analysis_name'].unique():
    print(crop)
    chems_count_ = df_.loc[df_['analysis_name'] == crop]
    print(chems_count_)
    crop_chem_unit_df = pd.DataFrame()
    for crop in df_['analysis_name'].unique():
        print(crop)
        chems_count_ = df_.loc[df_['analysis_name'] == crop]
        duplicates_chem = (chems_count_[chems_count_.duplicated(subset='chemical_name')])
        temp_crop_chem_unit_df = pd.DataFrame()
        for index,row in duplicates_chem.iterrows():
            chemical = row['chemical_name']
            chems_count_2 = chems_count_.loc[chems_count_['chemical_name'] == chemical ]
            unit = chems_count_2.loc[chems_count_2[0] == max(chems_count_2[0])]['unit_name'].values[0]
            _ = pd.DataFrame({"crop":[crop],"chemical_name":[chemical], "unit_name":[unit]})
            temp_crop_chem_unit_df = pd.concat([temp_crop_chem_unit_df, _])
        crop_chem_unit_df = pd.concat([crop_chem_unit_df, temp_crop_chem_unit_df])
crop_chem_unit_df.to_csv("output/soil_unit_per_chemical_decision.csv")

Complete Soil Analysis
              analysis_name           chemical_name unit_name    0
0    Complete Soil Analysis                    iron       ppm  850
1    Complete Soil Analysis                  sodium       ppm  843
2    Complete Soil Analysis                    zinc       ppm  843
3    Complete Soil Analysis                     cec  meq/100g  841
4    Complete Soil Analysis                 sulphur       ppm  835
5    Complete Soil Analysis                   boron       ppm  832
6    Complete Soil Analysis               potassium       ppm  831
7    Complete Soil Analysis               manganese       ppm  829
8    Complete Soil Analysis                 calcium       ppm  827
9    Complete Soil Analysis               magnesium       ppm  827
10   Complete Soil Analysis               aluminium       ppm  825
11   Complete Soil Analysis                      ph      None  822
12   Complete Soil Analysis                  copper       ppm  821
13   Complete Soil Analysis            

In [57]:
crop_chem_unit_df.crop.nunique()

17

In [58]:
df = pd.DataFrame()
for index, row in crop_chem_unit_df.iterrows():
    crop = row['crop']
    chemical = row['chemical_name']
    unit = row['unit_name']
    temp_df = soil_df.loc[(soil_df['analysis_name'] == crop) & (soil_df['chemical_name'] == chemical) & (soil_df['unit_name'] == unit)]
    df = pd.concat([df, temp_df])

In [59]:
df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
7,7,3,3,CF045SA0784,2012-07-09 00:00:00.000,Complete Soil Analysis,ec_salts,61.0,uS/cm
58,58,29,29,CF045SA0786,2012-07-09 00:00:00.000,Complete Soil Analysis,ec_salts,240.0,uS/cm
90,90,45,45,CF045SA0787,2012-07-09 00:00:00.000,Complete Soil Analysis,ec_salts,30.0,uS/cm
122,122,61,61,CF045SA0788,2012-07-09 00:00:00.000,Complete Soil Analysis,ec_salts,119.0,uS/cm
177,177,95,95,CF045SA0797,2012-07-09 00:00:00.000,Complete Soil Analysis,ec_salts,39.0,uS/cm
...,...,...,...,...,...,...,...,...,...
1104,1104,696,696,CW015SA0046,2012-03-05 00:00:00.000,Heavy Metals Analysis,aluminium,639.0,ppm
1189,1189,713,713,CW015SA0047,2012-03-05 00:00:00.000,Heavy Metals Analysis,aluminium,254.0,ppm
1239,1239,723,723,CW015SA0048,2012-03-05 00:00:00.000,Heavy Metals Analysis,aluminium,923.0,ppm
13717,13717,7126,7126,CW015SA0043,2012-02-29 00:00:00.000,Heavy Metals Analysis,aluminium,1395.0,ppm


In [60]:
df.to_csv("output/soil_analysis_cleaned.csv",chunksize=1000)

In [61]:
for i in df["analysis_name"]:
    print(i)

Complete Soil Analysis
Complete Soil Analysis
Complete Soil Analysis
Complete Soil Analysis
Complete Soil Analysis
Complete Soil Analysis
Complete Soil Analysis
Complete Soil Analysis
Complete Soil Analysis
Complete Soil Analysis
Complete Soil Analysis
Complete Soil Analysis
Complete Soil Analysis
Complete Soil Analysis
Complete Soil Analysis
Complete Soil Analysis
Complete Soil Analysis
Complete Soil Analysis
Complete Soil Analysis
Complete Soil Analysis
Complete Soil Analysis
Complete Soil Analysis
Complete Soil Analysis
Complete Soil Analysis
Complete Soil Analysis
Complete Soil Analysis
Complete Soil Analysis
Complete Soil Analysis
Complete Soil Analysis
Complete Soil Analysis
Complete Soil Analysis
Complete Soil Analysis
Complete Soil Analysis
Complete Soil Analysis
Complete Soil Analysis
Complete Soil Analysis
Complete Soil Analysis
Complete Soil Analysis
Complete Soil Analysis
Complete Soil Analysis
Complete Soil Analysis
Complete Soil Analysis
Complete Soil Analysis
Complete So

In [26]:
os.getcwd()

'D:\\Cropnuts\\DSML158\\SoilAnalysis'